<a href="https://colab.research.google.com/github/hfontoh/AIML/blob/main/Capstone%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [325]:
# Import the necessary files

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split #creating train and test splits
from sklearn.metrics import confusion_matrix, classification_report # model performance for logistic
from sklearn.linear_model import LinearRegression


import statsmodels.api as sm #regression models
import statsmodels.formula.api as smf # This allows you to use formulas inside statmodels functions

%matplotlib inline

In [326]:
#  Load dataset

df = pd.read_csv("/content/drive/MyDrive/Capstone1/2020_bexar.csv")
df.head().T

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
activity_year,2020,2020,2020,2020,2020
lei,254900HA4DQWAE0W3342,254900HA4DQWAE0W3342,254900HA4DQWAE0W3342,254900HA4DQWAE0W3342,254900HA4DQWAE0W3342
derived_msa_md,41700,41700,41700,41700,41700
state_code,TX,TX,TX,TX,TX
county_code,48029,48029,48029,48029,48029
...,...,...,...,...,...
ffiec_msa_md_median_family_income,72200,72200,72200,72200,72200
tract_to_msa_income_percentage,163,138,109,166,70
tract_owner_occupied_units,2331,1558,1020,2193,1187
tract_one_to_four_family_homes,2763,1968,1583,2564,2012


In [185]:
#df_1= df[['census_tract','action_taken', 'loan_type',  'loan_purpose', 'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate',  'total_loan_costs', 'origination_charges',
 #         'loan_term',  'property_value', 'construction_method',  'occupancy_type', 'total_units', 'income',  'debt_to_income_ratio']]

In [327]:
df_1= df[[ 'loan_amount', 'property_value', 'interest_rate','loan_term', 'income',  'debt_to_income_ratio','action_taken']]

In [328]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145273 entries, 0 to 145272
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amount           145273 non-null  int64  
 1   property_value        104284 non-null  object 
 2   interest_rate         97911 non-null   object 
 3   loan_term             144425 non-null  object 
 4   income                108457 non-null  float64
 5   debt_to_income_ratio  75451 non-null   object 
 6   action_taken          145273 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 7.8+ MB


In [329]:
# checking  for null values
df_1.isnull().sum()

loan_amount                 0
property_value          40989
interest_rate           47362
loan_term                 848
income                  36816
debt_to_income_ratio    69822
action_taken                0
dtype: int64

In [330]:
df_1['action_taken'].value_counts()

1    71316
6    21456
4    21099
3    17580
5     9037
2     4737
8       35
7       13
Name: action_taken, dtype: int64

In [331]:
# create binary data for action taken. 1 for loan approved and 0 for loan denied.
df_1['loan_status'] = np.where(df_1['action_taken'] <= 2, 1, 0)

# check  for categorical values : 1 for loan approved and 0 for loan denied
df_1['loan_status'].value_counts()  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1    76053
0    69220
Name: loan_status, dtype: int64

In [332]:

df_1['loan_term'].value_counts()

360    94611
180    15034
360    10344
240     5961
120     2274
       ...  
417        1
212        1
407        1
151        1
6.0        1
Name: loan_term, Length: 560, dtype: int64

In [333]:
#fill missing values  for numeric terms with their mean

df_1['loan_amount'].fillna(df_1['loan_amount'].mean(), inplace=True)

df_1['income'].fillna(df_1['income'].mean(), inplace=True)


df_1['loan_term'] =pd.to_numeric(df_1['loan_term'], errors='coerce')
k= df_1['loan_term'].mean()
df_1['loan_term'].fillna(k, inplace=True)

df_1['property_value'] =pd.to_numeric(df_1['property_value'], errors='coerce')
k= df_1['property_value'].mean()
df_1['property_value'].fillna(k, inplace=True)

df_1['interest_rate'] =pd.to_numeric(df_1['interest_rate'], errors='coerce')
k= df_1['interest_rate'].mean()
df_1['interest_rate'].fillna(k, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [334]:
#create  binary data  for debt to income ratio .  1 or ratios less than 36% 

def a_function(a):
  list= ['<20%', '20%-<30%', '30%-<36%']
  if a in list:
    res ='1'
  else:
   res='0'
  return res

df_1['debt_to_income_ratio'] = df_1['debt_to_income_ratio'].astype(str).map(a_function) 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [335]:
#check income debt_to_income_ratio
df_1['debt_to_income_ratio'].value_counts()

0    116050
1     29223
Name: debt_to_income_ratio, dtype: int64

In [336]:
df_1.isnull().sum()

loan_amount             0
property_value          0
interest_rate           0
loan_term               0
income                  0
debt_to_income_ratio    0
action_taken            0
loan_status             0
dtype: int64

In [338]:
del df_1['action_taken']
df_1.head()

,loan_amount,property_value,interest_rate,loan_term,income,debt_to_income_ratio,loan_status
0,335000,425000.0,3.875,360.0,345.000000,0,0
1,325000,425000.0,3.000,360.0,294.000000,0,0
2,215000,215000.0,2.250,360.0,846.143753,0,0
3,315000,475000.0,3.000,360.0,225.000000,0,0
4,185000,185000.0,4.000,360.0,95.000000,0,0


**split into labels/ target    training set /test sets**

In [345]:
#labels and target
X = df_1.drop('loan_status', axis=1)
y= df_1['loan_status']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

**Features Scaling**

In [346]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**TRAINING MODELS**

**RANDOMFOREST CLASSIFIER**

In [354]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=20, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [355]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[17811  2963]
 [ 2112 20696]]
              precision    recall  f1-score   support

           0       0.89      0.86      0.88     20774
           1       0.87      0.91      0.89     22808

    accuracy                           0.88     43582
   macro avg       0.88      0.88      0.88     43582
weighted avg       0.88      0.88      0.88     43582

0.8835528429168005


**XGBOOST XGBCLASSIFIER**

In [356]:
from xgboost import XGBClassifier

In [357]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [360]:
score = xgb_clf.score(X_test, y_test)
print(score)

0.8926391629571842


**KNN Classifier**

In [361]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [364]:
y_pred = classifier.predict(X_test)

In [365]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[17513  3261]
 [ 1759 21049]]
              precision    recall  f1-score   support

           0       0.91      0.84      0.87     20774
           1       0.87      0.92      0.89     22808

    accuracy                           0.88     43582
   macro avg       0.89      0.88      0.88     43582
weighted avg       0.89      0.88      0.88     43582



**LOGISTIC REGRESSION MODEL**

In [366]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

#Train the model and create predictions
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [367]:
#Calculate performance metrics

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.88      0.71     20774
           1       0.80      0.44      0.57     22808

    accuracy                           0.65     43582
   macro avg       0.70      0.66      0.64     43582
weighted avg       0.70      0.65      0.64     43582



In [369]:
#Generate a confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[18334  2440]
 [12738 10070]]
